# Mesh Mosaics

A mosaic is a custom collection of perspectives for the same scene. With ipyniivue, you use the `set_slice_mosaic_string` to define each tile for a lightbox view. Note that the load_meshes() command is asynchronous, so we need to use the on_mesh_loaded() to set the mosaic string after the mesh is available.

This Jupyter notebook mirrors the [mesh mosaic web page](https://niivue.com/demos/features/mosaics2.mesh.html).

**Note boggle atlas will require upgrade to [NiiVue 0.66](https://github.com/niivue/niivue/issues/1455)**

## Setup and Imports

In [ ]:
from pathlib import Path

import ipywidgets as widgets

import ipyniivue
from ipyniivue import NiiVue, SliceType, download_dataset

## Download Data

In [ ]:
BASE_API_URL = "https://niivue.com/demos/images/"
DATA_FOLDER = Path(ipyniivue.__file__).parent / "images"

download_dataset(
    BASE_API_URL,
    DATA_FOLDER,
    files=[
        "lh.pial",
        "lh.curv",
        "boggle.lh.annot",
    ],
)

## Initialize Viewer

In [ ]:
nv = NiiVue(
    show_3d_crosshair=True,
    back_color=(1, 1, 1, 1),
)

nv.set_slice_type(SliceType.RENDER)
nv.opts.is_colorbar = True
nv.opts.show_legend = False

## Define Layer Indices

In [ ]:
kCurvLayer = 0
kAtlasLayer = 1
kStatLayer = 2

## Load Mesh with Layers

In [ ]:
mesh_layers = [
    {
        "path": DATA_FOLDER / "lh.curv",
        "colormap": "gray",
        "cal_min": 0.49,
        "cal_max": 0.51,
        "opacity": 0.5,
    },
    {
        "path": DATA_FOLDER / "boggle.lh.annot",
        "opacity": 0.5,
    },
    {
        "path": DATA_FOLDER / "boggle.lh.annot",
        "opacity": 0.6,
    },
]

nv.load_meshes(
    [
        {
            "path": DATA_FOLDER / "lh.pial",
            "layers": mesh_layers,
        },
    ]
)

nv.set_clip_plane(-0.1, 270, 0)

## Configure Layers on Load

In [ ]:
@nv.on_mesh_loaded
def on_mesh_loaded(mesh):
    """Handle event after mesh is loaded and ready."""
    nv.meshes[0].layers[kStatLayer].cal_min = 2.3
    nv.meshes[0].layers[kStatLayer].cal_max = 5
    nv.meshes[0].layers[kStatLayer].colormap = "warm"
    nv.meshes[0].layers[kStatLayer].colormap_negative = "winter"
    nv.meshes[0].layers[kStatLayer].use_negative_cmap = True
    
    vals = [
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -3, 0, 0, 0, 0, 0, 0,
        0, -5, 0, 0, 7, 0, 0, 4, 7, 0, 0, 0, 0, 0, 0, 0,
    ]
    nv.meshes[0].layers[kStatLayer].atlas_values = vals
    
    nv.set_mesh_shader(nv.meshes[0].id, "Rim")
    nv.meshes[0].layers[kCurvLayer].colorbar_visible = False
    nv.meshes[0].layers[kAtlasLayer].colorbar_visible = True
    nv.opts.slice_mosaic_string = "A R 0 R -0 S R 0 R -0 C R 0 R -0"

## Create Interactive Controls

In [ ]:
curv_slider = widgets.IntSlider(
    value=50, min=1, max=100, description="Curvature", readout=False
)


def on_curv_change(change):
    """Set curve transparency."""
    if change["new"] is not None:
        nv.set_mesh_layer_property(
            mesh_id=nv.meshes[0].id,
            layer_index=kCurvLayer,
            attribute="opacity",
            value=change["new"] * 0.01,
        )


curv_slider.observe(on_curv_change, names="value")

In [ ]:
atlas_slider = widgets.IntSlider(
    value=50, min=1, max=100, description="Atlas", readout=False
)


def on_atlas_change(change):
    """Set atlas transparency."""
    if change["new"] is not None:
        nv.set_mesh_layer_property(
            mesh_id=nv.meshes[0].id,
            layer_index=kAtlasLayer,
            attribute="opacity",
            value=change["new"] * 0.01,
        )


atlas_slider.observe(on_atlas_change, names="value")

In [ ]:
stat_slider = widgets.IntSlider(
    value=60, min=1, max=100, description="Stats", readout=False
)


def on_stat_change(change):
    """Set stat transparency."""
    if change["new"] is not None:
        nv.set_mesh_layer_property(
            mesh_id=nv.meshes[0].id,
            layer_index=kStatLayer,
            attribute="opacity",
            value=change["new"] * 0.01,
        )


stat_slider.observe(on_stat_change, names="value")

In [ ]:
border_options = ["Dark border", "Transparent border", "No border", "Opaque border"]
border_dropdown = widgets.Dropdown(
    options=border_options,
    value="No border",
    description="Border",
)


def on_border_change(change):
    """Set mesh border style."""
    value_name = change["new"]
    borderValue = 0.0  # Default for "No border"

    if value_name == "Dark border":
        borderValue = -0.01
    elif value_name == "Transparent border":
        borderValue = 0.01
    elif value_name == "Opaque border":
        borderValue = 1.0

    nv.set_mesh_layer_property(
        nv.meshes[0].id, kAtlasLayer, "outline_border", borderValue
    )


border_dropdown.observe(on_border_change, names="value")

## Display Viewer

In [ ]:
widgets.VBox(
    [
        widgets.HBox([curv_slider, atlas_slider, border_dropdown, stat_slider]),
        nv,
    ]
)